In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My\ Drive/PRML-2020/Data\ Contest

/content/drive/My Drive/PRML-2020/Data Contest


In [ ]:
!pip install geopandas
!pip install geopy

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import pickle as pkl
from geopy.geocoders import Nominatim
from sklearn.decomposition import PCA

# Tours information

In [ ]:
tour_dict = pkl.load(open('./Required data/tours_list.pkl', 'rb'))
tour_ids = tour_dict.keys()
biker_dict = pkl.load(open('./Required data/bikers_list.pkl', 'rb'))
biker_ids = biker_dict.keys()

In [ ]:
tours_info = pd.read_csv('./dataset/tours.csv')
tours_info.index = tours_info['tour_id']
tours_info.drop('tour_id', axis=1, inplace=True)
tour_ids_not_reqd = list(set(tours_info.index) - set(tour_ids))
tours_info.drop(tour_ids_not_reqd, axis=0, inplace=True)
display(tours_info)

,biker_id,tour_date,city,state,pincode,country,latitude,longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,w32,...,w62,w63,w64,w65,w66,w67,w68,w69,w70,w71,w72,w73,w74,w75,w76,w77,w78,w79,w80,w81,w82,w83,w84,w85,w86,w87,w88,w89,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99,w100,w_other
tour_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
VX4921758,DG47864012,31-10-2012,NaN,NaN,NaN,NaN,NaN,NaN,2,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9
RT4999119,DE76440521,03-11-2012,NaN,NaN,NaN,NaN,NaN,NaN,2,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
SY28440935,FB7514445,05-11-2012,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
RU82345152,HI1585781,30-10-2012,NaN,NaN,NaN,NaN,NaN,NaN,1,0,2,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
QP51165850,BA16098580,27-09-2012,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,0,2,0,0,1,...,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SQ19357029,DD18624521,31-10-2012,London,NaN,NaN,United Kingdom,51.481,-0.191,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
RV89482290,IG2114700,01-08-2012,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,2,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
SS23611071,CJ60049566,15-12-2012,Los Angeles,CA,NaN,United States,34.034,-118.206,2,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,22


In [ ]:
print("No of uniques cities : {0}".format(len(tours_info['city'].unique())))
print("No of uniques states : {0}".format(len(tours_info['state'].unique())))
print("No of uniques countries : {0}".format(len(tours_info['country'].unique())))

No of uniques cities : 1972
No of uniques states : 134
No of uniques countries : 119


In [ ]:
tour_date = []
tour_latitude = []
tour_longitude = []
host_id = []
locator = Nominatim(user_agent='myGeocoder')
format = '%d-%m-%Y'

for tour_id in tqdm(tours_info.index):
    date = tours_info.loc[tour_id]['tour_date']
    cty = tours_info.loc[tour_id]['city']
    stt = tours_info.loc[tour_id]['state']
    ctry = tours_info.loc[tour_id]['country']
    ltt = tours_info.loc[tour_id]['latitude']
    lgt = tours_info.loc[tour_id]['longitude']
    try:
        host = tours_info.loc[tour_id]['biker_id']
        host_id.append(biker_dict[host])
    except:
        host_id.append(len(biker_ids))

    address = ''
    if isinstance(cty, str):
        address += (cty + ',')
    if isinstance(stt, str):
        address += (stt + ',')
    if isinstance(ctry, str):
        address += ctry

    if np.isnan(ltt) or np.isnan(lgt):
        if address:
            try:
                loc = locator.geocode(address)
                tour_latitude.append(loc.latitude)
                tour_longitude.append(loc.longitude)
            except:
                tour_latitude.append(ltt)
                tour_longitude.append(lgt)

        else:
            tour_latitude.append(ltt)
            tour_longitude.append(lgt)
    else:
        tour_latitude.append(ltt)
        tour_longitude.append(lgt)

    try:
        dt = datetime.strptime(date, format)
        dt_int = 10000*dt.year + 100*dt.month + dt.day
        tour_date.append(dt_int)
    except:
        tour_date.append(0)

In [12]:
tours_info['latitude'] = tour_latitude
tours_info['longitude'] = tour_longitude
tours_info['tour_date'] = tour_date
tours_info['host_id'] = host_id

tours_info_encoded = tours_info.drop(['city', 'state', 'country', 'pincode', 'biker_id'], axis=1)
tours_info_encoded.fillna(0, inplace=True)
tours_info_encoded.sort_index(inplace=True)
display(tours_info_encoded)

,tour_date,latitude,longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,w32,w33,w34,w35,w36,w37,...,w63,w64,w65,w66,w67,w68,w69,w70,w71,w72,w73,w74,w75,w76,w77,w78,w79,w80,w81,w82,w83,w84,w85,w86,w87,w88,w89,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99,w100,w_other,host_id
tour_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
QP00018501,20121214,26.120,-80.145,4,24,2,4,4,3,7,3,10,1,0,2,5,3,3,2,2,4,12,0,1,1,0,4,0,0,2,1,0,4,0,0,12,20,5,0,6,...,0,0,0,0,0,0,2,1,0,1,0,17,0,5,0,1,1,2,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,2,0,0,187,38209
QP0022787,20120714,0.000,0.000,2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,38209
QP00265896,20121126,3.565,98.647,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,107,38209
QP00453597,20121105,0.000,0.000,1,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,12,38209
QP00481836,20121102,0.000,0.000,1,0,0,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,261,38209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YY9643248,20121026,-6.198,106.815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,38209
YY9665321,20121006,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,38209
YY9690702,20120925,43.665,-79.374,3,0,0,0,0,4,0,0,0,0,0,0,1,0,0,1,1,0,1,0,2,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,52,38209


In [13]:
tours_info_encoded.to_csv('./Required data/tours_info.csv')